In [1]:
pip install jupyter-dash -q

     |████████████████████████████████| 7.3 MB 5.7 MB/s 
     |████████████████████████████████| 23.9 MB 13 kB/s 
     |████████████████████████████████| 357 kB 51.7 MB/s 


In [2]:
from jupyter_dash import JupyterDash  # pip install dash
#import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Output, Input
import pandas as pd  # pip install pandas
import plotly.express as px
import math
from dash import no_update

from urllib.request import urlopen
import json

############## LOADING APP DATA

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

#state codes
abbr = pd.read_csv("https://raw.githubusercontent.com/xroopnar/DSW2021/main/data/abbr-name.csv",index_col=False,header=None)
abbr.columns = ["state","location"]

#for the county-level state vaccination percentages
us = pd.read_csv("https://raw.githubusercontent.com/xroopnar/DSW2021/main/data/us_vaccine_9month.csv",index_col=0,dtype={"FIPS":str})

#for comparing new case count to vaccination percentage in bubble plot over time 
#cases = pd.read_csv("https://raw.githubusercontent.com/xroopnar/DSW2021/main/data/us_9month_covid_cases.csv",index_col=0)
#vacc = pd.read_csv("https://raw.githubusercontent.com/xroopnar/DSW2021/main/data/us_state_vaccinations.csv",index_col=False)
#vacc = vacc.merge(abbr,on="location",how="left")

#worldwide country covid cases
#world = pd.read_csv("https://raw.githubusercontent.com/xroopnar/DSW2021/main/data/WHO%20COVID-19%20global%20table%20data%20September%2011th%202021%20at%205.52.31%20PM.csv",index_col=False)

app = JupyterDash(__name__)

############### APP LAYOUT 

app.layout = html.Div([
    html.H1("COVID-19 Vaccination Tracker"),
    html.Div([
        dcc.Dropdown(
            id='ok-dropdown',
            value=us.columns[4],
            #clearable=False,
            options=[
                {'label': x, 'value': x}
                for x in us.columns[4:]
            ]),
        dcc.Dropdown(id="ok-date",
            value = us.Date.unique()[0],
            #clearable=False,
            options=[
                    {"label": x,"value": x}
                    for x in us.Date.unique()
            ]),
        dcc.Dropdown(id="state-map-dropdown",
            value = "OK",
            #clearable=False,
            options=[
                    {"label": x,"value": x}
                    for x in us.Recip_State.sort_values().unique()
            ])        
        ],style={'width': '30%', 'display': 'inline-block'}),
    dcc.Graph(id="ok-map"),
    html.Div([dcc.Dropdown(
        id="state-time-dropdown",
        value="OK",
        options=[
                 {"label":value,"value":value}
                 for value in us.Recip_State.unique()
                ]
        )],style={'width': '30%', 'display': 'inline-block'}),
    dcc.Graph(id="state-bar")
])

@app.callback(Output('ok-map', 'figure'),
              Input('ok-dropdown', 'value'),
              Input("ok-date","value"),
              Input("state-map-dropdown","value"))
def update_us_map(value,date,state):

  fig = px.choropleth(us[(us.Date==date) & (us.Recip_State==state)], geojson=counties, locations='FIPS', color=value,
                           color_continuous_scale="Viridis",
                           range_color=(0, 100),
                           scope="usa",
                           #labels={"Series_Complete_Pop_Pct":'Percent Vaccinated',"Recip_County":"County","Recip_State":"State"},
                           hover_data={"FIPS":False,"Recip_County":True,"Recip_State":True},
                           #use fitbounds when looking at a singular state instead of entire USA
                           #otherwise exclude fitbounds
                           fitbounds="locations",
                           title="Hello world"
                          )

  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},transition_duration=500)

  return(fig)

@app.callback(Output("state-bar","figure"),
              Input("state-time-dropdown","value"))
def update_state_time(value):

  fig = px.line(us[us.Recip_State==value].sort_values(by="Date",ascending=True),x="Date",y="Series_Complete_Pop_Pct",line_group="Recip_County",color="Recip_County")
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},transition_duration=500)
  return(fig)

app.run_server(mode='inline', port=8030)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  after removing the cwd from sys.path.


<IPython.core.display.Javascript object>